In [6]:
import pandas as pd
import string
from collections import defaultdict as dd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))
import re
from tqdm import tqdm as tqdm

In [7]:
df = pd.read_csv("data/fitocracy_posts.csv")

In [13]:
len(df[df['op_gender'] == "M"]), len(df[df['op_gender'] == "W"]), len(df)

(173738, 144797, 318535)

In [14]:
df

,op_id,op_gender,post_id,post_text
0,102,W,0,Thanks for the follow! I followed back :) I ...
1,104,W,1,I've decided I have this crazy goal of running...
2,104,W,2,"Umm, yeah, so those are assisted pull-ups. Bef..."
3,117,M,3,dam gurl lookin mad tone in dat pp holla bb
4,117,M,4,What's up there bear mode?
5,117,M,5,Where have you been?
6,117,M,6,Incidentally -- you planning to compete at tha...
7,117,M,7,"I have pains in many places, and being somethi..."
8,117,M,8,God damn but you are an enthusiastic woman.
9,117,M,9,"I just ate a chicken, got a half pound of gree..."


In [16]:
text_tag = 'post_text'

In [57]:
unigrams = dd(dict)
exclude = set(string.punctuation)

for index, row in tqdm(df.iterrows()):
    
    text = row[text_tag].lower()
    label = row['op_gender']
    
    tokens = set(word_tokenize(str(text)))
    tokens = [w for w in tokens if not w in stopset]
    
    for word in tokens:
        
        if (('#' not in word) and 
            ('www.' not in word) and 
            ('.com' not in word) and
            ('#' not in word) and 
            ('@' not in word) and
            (len(word)<15) and 
            ('http:' not in word) and 
            not (re.sub('[^A-Za-z0-9]+', '', word).isdigit())):
            
           # word = ''.join(ch for ch in word if ch not in exclude)
            
            if label in unigrams[word]:
                unigrams[word][label] += 1
            else:
                unigrams[word][label] = 1

318535it [02:05, 2542.39it/s]


In [58]:
salience = dd(dict)
reverse_dict = {'W': 'M', 'M': 'W'}
l = 1
for word in unigrams:
    
    for label in ['W', 'M']:
        if label not in unigrams[word]:
            unigrams[word][label] = 0
    
    for label in unigrams[word]:
        reverse = reverse_dict[label]
        salience[word][label] = (unigrams[word][label] + l) / (unigrams[word][reverse] + l)

In [59]:
salience = dict(salience)

In [60]:
save_object(salience, "salience.pkl")
save_object(unigrams, "unigrams.pkl")

In [61]:
len(salience)

84269

In [62]:
salience

{'followed': {'W': 1.1365853658536584, 'M': 0.8798283261802575},
 "'s": {'W': 0.8739432493513016, 'M': 1.1442390575615362},
 'kick': {'W': 1.049222797927461, 'M': 0.9530864197530864},
 ':': {'W': 1.4076625529563456, 'M': 0.7103975295072099},
 'let': {'W': 0.8130263896687254, 'M': 1.2299723756906078},
 'wan': {'W': 0.7958236658932715, 'M': 1.2565597667638484},
 'follow': {'W': 0.8867801756859343, 'M': 1.127675186498716},
 'butt': {'W': 1.7635782747603834, 'M': 0.5670289855072463},
 ',': {'W': 0.7651279758578294, 'M': 1.3069709010167114},
 'na': {'W': 0.7190946024376088, 'M': 1.390637610976594},
 'thanks': {'W': 0.8399225040866986, 'M': 1.1905860304187992},
 'back': {'W': 0.8440752812532857, 'M': 1.1847284504235176},
 ')': {'W': 1.4122531988814506, 'M': 0.7080883235329413},
 '!': {'W': 1.0075854656690504, 'M': 0.9924716404439066},
 "'ve": {'W': 0.9524661893396977, 'M': 1.0499060346627689},
 'running': {'W': 1.09697933227345, 'M': 0.9115942028985508},
 'triathlon': {'W': 0.801652892561983

In [98]:
sal = [(key,value['M'],value['W'], max(value['M'], value['W']), unigrams[key]) for key, value in salience.items()]

In [99]:
[(k[0], k[3], k[4]) for k in sorted(sal, key= lambda val: val[1], reverse=True)]

[('brotha', 69.0, {'M': 68, 'W': 0}),
 ('jr.', 26.0, {'M': 25, 'W': 0}),
 ('thxs', 25.0, {'M': 24, 'W': 0}),
 ('brandon', 24.0, {'M': 119, 'W': 4}),
 ('cunt', 23.0, {'M': 22, 'W': 0}),
 ('faggot', 22.0, {'M': 21, 'W': 0}),
 ('fatmouse', 22.0, {'M': 21, 'W': 0}),
 ('hombre', 20.0, {'M': 19, 'W': 0}),
 ('gay', 18.25, {'M': 72, 'W': 3}),
 ('gymnastic', 18.0, {'M': 17, 'W': 0}),
 ('gripz', 18.0, {'M': 17, 'W': 0}),
 ('martin', 17.0, {'M': 16, 'W': 0}),
 ('ranks', 16.0, {'M': 15, 'W': 0}),
 ('mazy', 16.0, {'M': 15, 'W': 0}),
 ('muscle-ups', 16.0, {'M': 15, 'W': 0}),
 ('pistol', 15.0, {'M': 14, 'W': 0}),
 ('everypony', 15.0, {'M': 14, 'W': 0}),
 ('mate', 14.953125, {'M': 956, 'W': 63}),
 ('1000lbs', 14.0, {'M': 13, 'W': 0}),
 ('rabidmoose', 14.0, {'M': 13, 'W': 0}),
 ('***', 14.0, {'W': 1, 'M': 27}),
 ('traveled', 14.0, {'M': 13, 'W': 0}),
 ('candito', 14.0, {'M': 13, 'W': 0}),
 ('smolov', 13.5, {'M': 80, 'W': 5}),
 ('obliged', 13.0, {'M': 12, 'W': 0}),
 ('bruh', 13.0, {'M': 25, 'W': 1}),
 (

In [100]:
[(k[0], k[3], k[4]) for k in sorted(sal, key= lambda val: val[2], reverse=True)]

[('angels', 248.0, {'W': 991, 'M': 3}),
 ('barbie', 91.8, {'W': 4130, 'M': 44}),
 ('kixies', 78.0, {'W': 77, 'M': 0}),
 ('girls', 22.231638418079097, {'M': 176, 'W': 3934}),
 ('ravelry', 22.0, {'W': 21, 'M': 0}),
 ('blogilates', 20.0, {'W': 19, 'M': 0}),
 ('aerobics', 19.5, {'W': 38, 'M': 1}),
 ('bestie', 17.0, {'W': 16, 'M': 0}),
 ('eek', 16.5, {'W': 32, 'M': 1}),
 ('spa', 15.5, {'W': 30, 'M': 1}),
 ('lifters', 12.123456790123457, {'M': 80, 'W': 981}),
 ('yayyy', 12.0, {'W': 11, 'M': 0}),
 ('yippee', 12.0, {'W': 11, 'M': 0}),
 ("darlin'-", 12.0, {'W': 11, 'M': 0}),
 ('fitspo', 11.0, {'W': 10, 'M': 0}),
 ('allie', 11.0, {'W': 10, 'M': 0}),
 ('bellydancing', 11.0, {'W': 10, 'M': 0}),
 ('^-^', 11.0, {'W': 32, 'M': 2}),
 ('livefit', 11.0, {'W': 10, 'M': 0}),
 ("o'-'o", 11.0, {'W': 10, 'M': 0}),
 ('^_^best', 11.0, {'W': 10, 'M': 0}),
 ('athena', 10.0, {'W': 9, 'M': 0}),
 ('minions', 10.0, {'W': 9, 'M': 0}),
 ('fibromyalgia', 10.0, {'W': 19, 'M': 1}),
 ('hypermobility', 10.0, {'W': 9, 'M': 

In [ ]:
"3.14".replace('.','',1).isdigit()

In [ ]:
re.sub('[^A-Za-z0-9]+', '', "3.14")

# SENTENCE EMBEDS

In [17]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint
import matplotlib

import numpy as np
import torch

GLOVE_PATH = 'glove/glove.840B.300d.txt'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# make sure models.py is in the working directory
model = torch.load('infersent.allnli.pickle', map_location=lambda storage, loc: storage)

/usr/local/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'models.BLSTMEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [19]:
model.set_glove_path(GLOVE_PATH)

In [20]:
model.build_vocab_k_words(K=500000)

Vocab size : 500000


In [63]:
sentences = []
labels = []
max_it = 320000
for index, row in tqdm(df.iterrows()):
    text = row['post_text'].lower()
    label = row['op_gender']
    tokens = word_tokenize(str(text))
    new_text = " ".join(tokens)
    
    if index>max_it:
        break
    labels.append(label)
    sentences.append(new_text)
    
print(len(sentences))

318535it [01:51, 2862.71it/s]

318535


In [23]:
sentences[:5]

["thanks for the follow ! i followed back : ) i wan na kick some butt , too : ) let 's do it !",
 "i 've decided i have this crazy goal of running a triathlon .",
 "umm , yeah , so those are assisted pull-ups . before everyone starts thinking i 'm a badass .",
 'dam gurl lookin mad tone in dat pp holla bb',
 "what 's up there bear mode ?"]

In [24]:
embeddings = model.encode(sentences, bsize=128, tokenize=False, verbose=True)
print('nb sentences encoded : {0}'.format(len(embeddings)))

/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'ohmyholyyouravatarpic.jpg', '</s>']" (idx=37) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'sploooooosh', '</s>']" (idx=133) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'herro', 'everynyan', '</s>']" (idx=164) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'sup', '</s>']" (idx=188) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(senten

/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'yaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaay', '</s>']" (idx=78165) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'iiiiidontworkout', '</s>']" (idx=78175) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'bwahahahaha', '</s>']" (idx=78227) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'pammmmmm', '</s>']" (idx=78296) have glove vectors.                                Replacing by "</s>"..
  Replacing by "<

/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'hiii', '</s>']" (idx=134484) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', '*curtsy*', '</s>']" (idx=134582) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'homo', '</s>']" (idx=135246) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'omg', '</s>']" (idx=135540) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayom

/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'zzzzzzzzzzzzzzz', '</s>']" (idx=190776) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'xxx', '</s>']" (idx=191238) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'yo', '</s>']" (idx=191242) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'heelo', '</s>']" (idx=191263) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/

/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'xoxo', '</s>']" (idx=253365) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'bbb', '</s>']" (idx=253648) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'bully', '</s>']" (idx=253709) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', '@', 'cj_huntsmud', '</s>']" (idx=253961) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Use

/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', '143.5', '</s>']" (idx=317277) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'howdy', '</s>']" (idx=317599) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'maow', '</s>']" (idx=317604) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/Docs/independent/rt/rtgender/models.py:182: UserWarning: No words in "['<s>', 'sup', '</s>']" (idx=317771) have glove vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..'.format(sentences[i], i))
/Users/gbayomi/

Nb words kept : 5125840/6311084 (81.22 %)


/Users/gbayomi/Docs/independent/rt/rtgender/models.py:206: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  sentences[stidx:stidx + bsize]), volatile=True)


KeyError: '</s>'

In [25]:
import pickle

In [26]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as _input:
        return pickle.load(_input)

In [27]:
save_object(model, "infersent_model.pkl")

In [28]:
ml = load_object("infersent_model.pkl")

In [29]:
np.linalg.norm(ml.encode(['the cat eats.']))

/Users/gbayomi/Docs/independent/rt/rtgender/models.py:206: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  sentences[stidx:stidx + bsize]), volatile=True)


3.8170834

In [36]:
answers = [
    'I liked Titanic, it was my favorite this year.',
    'I think that Jazz is my favorite.',
    'I really like Arctic Monkeys.',
    'I like beans and rice.'
    'A good pasta is always the best.',
    'The movie is great.'
]

question = 'Which movies do you like?'

In [91]:
from sklearn.metrics.pairwise import cosine_similarity

def compare_to_all(sentence, sample, labels, k=1):
    
    sample_encoded = model.encode(sample, bsize=128, tokenize=False, verbose=True)
    
    _s = model.encode([sentence])
    _sim = cosine_similarity(_s, sample_encoded)
    
    sim = [v for v in _sim[0]]
    
    tups = list(zip(sample, labels, sim))
    tups.sort(key=lambda x: x[2], reverse=True)
    
    return tups[:k]

# RETRIEVE MODE

In [64]:
sentence, label = sentences[0], labels[0]
print(sentence, label)

thanks for the follow ! i followed back : ) i wan na kick some butt , too : ) let 's do it ! W


In [74]:
sal_sentence = [(tok, salience[tok][label]) for tok in word_tokenize(sentence) if 
                ((tok in salience) and (tok not in stopset) and (tok not in string.punctuation))]
sal_sentence.sort(reverse=True, key=lambda x: x[1])

In [75]:
sal_sentence

[('butt', 1.7635782747603834),
 ('followed', 1.1365853658536584),
 ('kick', 1.049222797927461),
 ('follow', 0.8867801756859343),
 ("'s", 0.8739432493513016),
 ('back', 0.8440752812532857),
 ('thanks', 0.8399225040866986),
 ('let', 0.8130263896687254),
 ('wan', 0.7958236658932715),
 ('na', 0.7190946024376088)]

In [93]:
sample = sentences[:10]
l_sample = labels[:10]

compare_to_all(sentence, sample, l_sample, k=5)     

Nb words kept : 226/279 (81.0 %)


/Users/gbayomi/Docs/independent/rt/rtgender/models.py:206: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  sentences[stidx:stidx + bsize]), volatile=True)


Speed : 7.01 sentences/s (cpu mode, bsize=128)


[("thanks for the follow ! i followed back : ) i wan na kick some butt , too : ) let 's do it !",
  'W',
  0.9747988),
 ("i have pains in many places , and being something of an idiot ca n't decide if 5/3/1 squat day actually kicked my ass or if i 'm so damn weak my body ca n't even cope with a couple half-assed cleans . either way i 'd like to take a moment right now to endorse both major medical marijuana reform in 2013 and the healing power of all-the-meat-in-my-fridge.fuark i 'm old .",
  'M',
  0.7481489),
 ('incidentally -- you planning to compete at that meet on the 16th ? i was telling a kid that works at my y about it and it turns out he and another guy that trains up here are going to be in it .',
  'M',
  0.7476131),
 ("umm , yeah , so those are assisted pull-ups . before everyone starts thinking i 'm a badass .",
  'W',
  0.7395567),
 ('i just ate a chicken , got a half pound of green beans and a half box of parmesan noodles inside me , and should still only be around 2500 

In [90]:
sample

["thanks for the follow ! i followed back : ) i wan na kick some butt , too : ) let 's do it !",
 "i 've decided i have this crazy goal of running a triathlon .",
 "umm , yeah , so those are assisted pull-ups . before everyone starts thinking i 'm a badass .",
 'dam gurl lookin mad tone in dat pp holla bb',
 "what 's up there bear mode ?",
 'where have you been ?',
 'incidentally -- you planning to compete at that meet on the 16th ? i was telling a kid that works at my y about it and it turns out he and another guy that trains up here are going to be in it .',
 "i have pains in many places , and being something of an idiot ca n't decide if 5/3/1 squat day actually kicked my ass or if i 'm so damn weak my body ca n't even cope with a couple half-assed cleans . either way i 'd like to take a moment right now to endorse both major medical marijuana reform in 2013 and the healing power of all-the-meat-in-my-fridge.fuark i 'm old .",
 'god damn but you are an enthusiastic woman .',
 'i just

In [94]:
sentence = "i have pains in many places , and being something of an idiot ca n't decide if 5/3/1 squat day actually kicked my ass or if i 'm so damn weak my body ca n't even cope with a couple half-assed cleans . either way i 'd like to take a moment right now to endorse both major medical marijuana reform in 2013 and the healing power of all-the-meat-in-my-fridge.fuark i 'm old ."

In [95]:
label = "M"

In [96]:
sal_sentence = [(tok, salience[tok][label]) for tok in word_tokenize(sentence) if 
                ((tok in salience) and (tok not in stopset) and (tok not in string.punctuation))]
sal_sentence.sort(reverse=True, key=lambda x: x[1])

In [97]:
sal_sentence

[('half-assed', 2.5),
 ('endorse', 2.5),
 ('cleans', 2.1515151515151514),
 ('squat', 2.0354131534569984),
 ('marijuana', 2.0),
 ('reform', 2.0),
 ('damn', 1.8738404452690167),
 ('power', 1.8076923076923077),
 ('cope', 1.6666666666666667),
 ('weak', 1.35),
 ('old', 1.33203125),
 ("'d", 1.2873462214411249),
 ('couple', 1.279475982532751),
 ('ass', 1.261501210653753),
 ('pains', 1.2608695652173914),
 ('like', 1.2244100640725113),
 ('something', 1.1909975669099757),
 ('either', 1.1561561561561562),
 ('take', 1.1422791638570464),
 ('many', 1.1127331711273316),
 ('way', 1.0974610974610974),
 ('medical', 1.0166666666666666),
 ('even', 1.0149597238204833),
 ('right', 1.0145656779661016),
 ("n't", 1.004855728826221),
 ("n't", 1.004855728826221),
 ('day', 1.0026455026455026),
 ('body', 0.992014742014742),
 ('idiot', 0.975),
 ('decide', 0.9689922480620154),
 ('actually', 0.9572553430821147),
 ('healing', 0.9529411764705882),
 ('moment', 0.9481481481481482),
 ('kicked', 0.9130434782608695),
 ("'m"

original

thanks for the follow ! i followed back : ) i wan na kick some butt , too : ) let 's do it ! -> W

changed

"thanks for the follow ! i followed back : ) i wan na kick some half-assed , too : ) let 's do it ! -> M